For this project you'll dig into a large amount of text and apply most of what you've covered in this unit and in the course so far.

First, pick a set of texts. This can be either a series of novels, chapters, or articles. Anything you'd like. It just has to have multiple entries of varying characteristics. At least 100 should be good. There should also be at least 10 different authors, but try to keep the texts related (either all on the same topic of from the same branch of literature - something to make classification a bit more difficult than obviously different subjects).

This capstone can be an extension of your NLP challenge if you wish to use the same corpus. If you found problems with that data set that limited your analysis, however, it may be worth using what you learned to choose a new corpus. Reserve 25% of your corpus as a test set.

The first technique is to create a series of clusters. Try several techniques and pick the one you think best represents your data. Make sure there is a narrative and reasoning around why you have chosen the given clusters. Are authors consistently grouped into the same cluster?

Next, perform some unsupervised feature generation and selection using the techniques covered in this unit and elsewhere in the course. Using those features then build models to attempt to classify your texts by author. Try different permutations of unsupervised and supervised techniques to see which combinations have the best performance.

Lastly return to your holdout group. Does your clustering on those members perform as you'd expect? Have your clusters remained stable or changed dramatically? What about your model? Is its performance consistent?

If there is a divergence in the relative stability of your model and your clusters, delve into why.

Your end result should be a write up of how clustering and modeling compare for classifying your texts. What are the advantages of each? Why would you want to use one over the other? Approximately 3-5 pages is a good length for your write up, and remember to include visuals to help tell your story!

In [1]:
# Set up data science environment.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import scipy
import spacy
import re
import warnings
from nltk.corpus import inaugural, stopwords
from sklearn.cluster import KMeans, MiniBatchKMeans, SpectralClustering
from sklearn.cluster import AffinityPropagation, MeanShift, estimate_bandwidth
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, normalize
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from typing import Dict

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format
sns.set_style('white')

# Suppress annoying harmless error.
warnings.filterwarnings(
    action='ignore',
    module='scipy',
    message='internal gelsd'
)

# Data Cleaning, Processing, and Language Parsing

In [2]:
# Create lists for files and presidents.
files = ["1789-Washington.txt",
         "1801-Jefferson.txt",
         "1861-Lincoln.txt",
         "1933-Roosevelt.txt",
         "1953-Eisenhower.txt",
         "1961-Kennedy.txt",
         "1981-Reagan.txt",
         "1989-Bush.txt",
         "1993-Clinton.txt",
         "2009-Obama.txt"]

presidents = ["washington",
              "jefferson",
              "lincoln",
              "fdr",
              "eisenhower",
              "kennedy",
              "reagan",
              "bush41",
              "clinton",
              "obama"]

# Control to make sure both lists are the same length.
assert len(files) == len(presidents)

In [3]:
# Quick function to open all files needed (and close them again).
docs = []
for file_name, president in zip(files, presidents):
    with open(f'./inaugural/{file_name}') as f:
        doc = f.read()
        docs.append((doc, president))

In [4]:
# Utility function to clean text.
def text_cleaner(text: str) -> str:
    """Function to strip all characters except letters in words."""
    text = re.sub(r'--', ' ', text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = re.sub("[\<].*?[\>]", "", text)
    text = ' '.join(text.split())
    return text

In [5]:
# Use text_cleaner on the docs, combine them into data frame (clean_docs).
clean_docs = []
for doc, pres in docs:
    clean_doc = text_cleaner(doc)
    clean_docs.append((clean_doc, pres))

In [6]:
# Iterate through each doc and print the first 1000 characters for inspection.
for doc, pres in clean_docs:
    print(doc[:1000], pres.upper()) 
    print()

Fellow-Citizens of the Senate and of the House of Representatives: Among the vicissitudes incident to life no event could have filled me with greater anxieties than that of which the notification was transmitted by your order, and received on the 14th day of the present month. On the one hand, I was summoned by my Country, whose voice I can never hear but with veneration and love, from a retreat which I had chosen with the fondest predilection, and, in my flattering hopes, with an immutable decision, as the asylum of my declining years a retreat which was rendered every day more necessary as well as more dear to me by the addition of habit to inclination, and of frequent interruptions in my health to the gradual waste committed on it by time. On the other hand, the magnitude and difficulty of the trust to which the voice of my country called me, being sufficient to awaken in the wisest and most experienced of her citizens a distrustful scrutiny into his qualifications, could not but ov

In [7]:
# Function to parse data.
nlp = spacy.load('en')
df_list = []


def nlp_text(text_file):
    doc = nlp(text_file)
    return doc

def sentences(doc_nlp, speaker):
    return [[sent.text, speaker] for sent in doc_nlp.sents]

  
def sentences_to_df(sents):
    return pd.DataFrame(sents)


for doc, pres in clean_docs:
    parsed = nlp_text(doc)
    sents = sentences(parsed, pres)
    df = sentences_to_df(sents)
    df_list.append(df)

In [8]:
# Combine each sentence data frame into one master data frame.
sent_df = pd.concat([*df_list])

In [9]:
# Rename columns.
sent_df.columns = ['sentence', 'President']

# Check the count of sents per President.
sent_df.President.value_counts()

bush41        145
lincoln       139
reagan        130
eisenhower    121
obama         113
fdr            86
clinton        82
kennedy        53
jefferson      42
washington     25
Name: President, dtype: int64

# Creating Features
## Bag of Words

In [10]:
# Splitting the data.
X = sent_df.sentence
y = sent_df.President
X_train_eval, X_holdout, y_train_eval, y_holdout = train_test_split(
    X, y, test_size=0.25, random_state=15)

In [11]:
# Splitting into train/eval/holdout groups.
X_train, X_eval, y_train, y_eval = train_test_split(
    X_train_eval, y_train_eval, test_size=0.25, random_state=15)

In [12]:
# Create base parameters dictionary.
base_param_dict = {'strip_accents': 'unicode',
                   'lowercase': True,
                   'stop_words': 'english'}

In [13]:
# Instantiate CountVectorizer.
bow = CountVectorizer(**base_param_dict)

In [14]:
# Convert X_train, X_test into dfs of bags of words.
_bow_train = bow.fit_transform(X_train)
_bow_eval = bow.transform(X_eval)
_bow_holdout = bow.transform(X_holdout)
assert len(X_train) == _bow_train.shape[0]  # df and sparse-matrix

# Find feature names.
feature_names = bow.get_feature_names()

# Sparse matrix to data frame.
X_train_bow = pd.DataFrame(_bow_train.toarray(), columns=feature_names)
X_eval_bow = pd.DataFrame(_bow_eval.toarray(), columns=feature_names)
X_holdout_bow = pd.DataFrame(_bow_holdout.toarray(), columns=feature_names)

## Tfidf

In [15]:
# Instantiate Tfidf.
tfidf = TfidfVectorizer(**base_param_dict)

In [16]:
# Convert X_train, X_test into dfs of tfidf values.
_tfidf_train = tfidf.fit_transform(X_train)
_tfidf_eval = tfidf.transform(X_eval)
_tfidf_holdout = tfidf.transform(X_holdout)
assert len(X_train) == _tfidf_train.shape[0]  # df and sparse-matrix

# Find feature names.
feature_names_tfidf = tfidf.get_feature_names()

# Set up data frames.
X_train_tfidf = pd.DataFrame(
    _tfidf_train.toarray(), columns=feature_names_tfidf)
X_eval_tfidf = pd.DataFrame(
    _tfidf_eval.toarray(), columns=feature_names_tfidf)
X_holdout_tfidf = pd.DataFrame(
    _tfidf_holdout.toarray(), columns=feature_names_tfidf)

# Clustering Models

In [17]:
# Instantiate MinMaxScaler. Create train/eval/holdout groups.
scaler = MinMaxScaler()
X_train_tfidf_scaled = scaler.fit_transform(X_train_tfidf)
print(len(X_train_tfidf_scaled))
X_eval_tfidf_scaled = scaler.transform(X_eval_tfidf)
X_holdout_tfidf_scaled = scaler.transform(X_holdout_tfidf)

526


In [18]:
# Clustering models
models = []
names = []
plot_nums = []
silhouettes = []
clust = []

for clusters in range(2,11):
    bandwidth = estimate_bandwidth(
        X_train_tfidf_scaled, quantile=0.2, n_samples=500)
    models.append((0, 'KMeans', KMeans(n_clusters=clusters,
                                   init='k-means++', random_state=42)))
    models.append((1, 'MiniBatch', MiniBatchKMeans(
        init='random', n_clusters=clusters, batch_size=500)))
    models.append((2, 'MeanShift', MeanShift(
        bandwidth=bandwidth, bin_seeding=True)))
    models.append((3, 'Spectral', SpectralClustering(n_clusters=clusters)))
    #models.append((4, 'Affinity', AffinityPropagation()))
    
for _, name, model in models:
    names.append(name)
    model.fit(X_train_tfidf_scaled)
    labels = model.labels_
    print(model)
    if len(set(labels)) > 1:
        ypred = model.fit_predict(X_train_tfidf_scaled)
        silhouette = metrics.silhouette_score(
            X_train_tfidf_scaled, labels, metric='euclidean')
        silhouettes.append(silhouette)
        #ax[plot_num].set_title(name)
        #plotting(plot_num, labels, ypred)
        print('clusters: {}\t silhouette: {}\n'.format(
            model.n_clusters, silhouette))
        print(name, '\n', pd.crosstab(ypred, labels), '\n')

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=42, tol=0.0001, verbose=0)
clusters: 2	 silhouette: 0.5439510271051321

KMeans 
 col_0  0    1
row_0        
0      1    0
1      0  525 

MiniBatchKMeans(batch_size=500, compute_labels=True, init='random',
        init_size=None, max_iter=100, max_no_improvement=10, n_clusters=2,
        n_init=3, random_state=None, reassignment_ratio=0.01, tol=0.0,
        verbose=0)
clusters: 2	 silhouette: 0.09495640752724352

MiniBatch 
 col_0  0    1
row_0        
0      1  524
1      0    1 

MeanShift(bandwidth=3.071450947055253, bin_seeding=True, cluster_all=True,
     min_bin_freq=1, n_jobs=None, seeds=None)
SpectralClustering(affinity='rbf', assign_labels='kmeans', coef0=1, degree=3,
          eigen_solver=None, eigen_tol=0.0, gamma=1.0, kernel_params=None,
          n_clusters=2, n_init=10, n_jobs=None, n_neighbors=10,
          random

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=8, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=42, tol=0.0001, verbose=0)
clusters: 8	 silhouette: -0.05594710095693154

KMeans 
 col_0  0  1  2  3  4    5  6  7
row_0                          
0      1  0  0  0  0    0  0  0
1      0  5  0  0  0    0  0  0
2      0  0  1  0  0    0  0  0
3      0  0  0  3  0    0  0  0
4      0  0  0  0  1    0  0  0
5      0  0  0  0  0  513  0  0
6      0  0  0  0  0    0  1  0
7      0  0  0  0  0    0  0  1 

MiniBatchKMeans(batch_size=500, compute_labels=True, init='random',
        init_size=None, max_iter=100, max_no_improvement=10, n_clusters=8,
        n_init=3, random_state=None, reassignment_ratio=0.01, tol=0.0,
        verbose=0)
clusters: 8	 silhouette: -0.03960830807412723

MiniBatch 
 col_0   0  1  2  3  4  5    6  7
row_0                           
0       0  0  0  0  0  0    1  0
1       0  0  0  0  0  0    9  0
2       1  0 

In [19]:
# Re-run KMeans and extract cluster information.
model = KMeans(n_clusters=2, random_state=15).fit(X_train_tfidf_scaled)

# Extract cluster assignments for each data point.
labels = model.labels_

X_train_tfidf['clusters'] = labels

In [20]:
X_train_tfidf_clusters = X_train_tfidf.groupby(
    ['clusters'], as_index=False).mean()
X_train_tfidf_clusters

,clusters,100,14th,1776,1787,1917,200,21st,abdicated,abhorring,...,write,written,wrong,year,yearn,years,yes,yields,young,zeal
0,0,0.000,0.000,0.000,0.000,0.000,0.008,0.000,0.000,0.000,...,0.000,0.032,0.000,0.000,0.010,0.018,0.000,0.000,0.000,0.000
1,1,0.001,0.001,0.001,0.001,0.001,0.000,0.001,0.001,0.001,...,0.001,0.002,0.002,0.001,0.000,0.002,0.003,0.001,0.004,0.001


In [21]:
cluster0 = X_train_tfidf_clusters[X_train_tfidf_clusters['clusters'] == 0]
cluster0.describe()

,clusters,100,14th,1776,1787,1917,200,21st,abdicated,abhorring,...,write,written,wrong,year,yearn,years,yes,yields,young,zeal
count,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
mean,0.000,0.000,0.000,0.000,0.000,0.000,0.008,0.000,0.000,0.000,...,0.000,0.032,0.000,0.000,0.010,0.018,0.000,0.000,0.000,0.000
std,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
min,0.000,0.000,0.000,0.000,0.000,0.000,0.008,0.000,0.000,0.000,...,0.000,0.032,0.000,0.000,0.010,0.018,0.000,0.000,0.000,0.000
25%,0.000,0.000,0.000,0.000,0.000,0.000,0.008,0.000,0.000,0.000,...,0.000,0.032,0.000,0.000,0.010,0.018,0.000,0.000,0.000,0.000
50%,0.000,0.000,0.000,0.000,0.000,0.000,0.008,0.000,0.000,0.000,...,0.000,0.032,0.000,0.000,0.010,0.018,0.000,0.000,0.000,0.000
75%,0.000,0.000,0.000,0.000,0.000,0.000,0.008,0.000,0.000,0.000,...,0.000,0.032,0.000,0.000,0.010,0.018,0.000,0.000,0.000,0.000
max,0.000,0.000,0.000,0.000,0.000,0.000,0.008,0.000,0.000,0.000,...,0.000,0.032,0.000,0.000,0.010,0.018,0.000,0.000,0.000,0.000


In [22]:
cluster1 = X_train_tfidf_clusters[X_train_tfidf_clusters['clusters'] == 1]
cluster1.describe()

,clusters,100,14th,1776,1787,1917,200,21st,abdicated,abhorring,...,write,written,wrong,year,yearn,years,yes,yields,young,zeal
count,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
mean,1.000,0.001,0.001,0.001,0.001,0.001,0.000,0.001,0.001,0.001,...,0.001,0.002,0.002,0.001,0.000,0.002,0.003,0.001,0.004,0.001
std,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
min,1.000,0.001,0.001,0.001,0.001,0.001,0.000,0.001,0.001,0.001,...,0.001,0.002,0.002,0.001,0.000,0.002,0.003,0.001,0.004,0.001
25%,1.000,0.001,0.001,0.001,0.001,0.001,0.000,0.001,0.001,0.001,...,0.001,0.002,0.002,0.001,0.000,0.002,0.003,0.001,0.004,0.001
50%,1.000,0.001,0.001,0.001,0.001,0.001,0.000,0.001,0.001,0.001,...,0.001,0.002,0.002,0.001,0.000,0.002,0.003,0.001,0.004,0.001
75%,1.000,0.001,0.001,0.001,0.001,0.001,0.000,0.001,0.001,0.001,...,0.001,0.002,0.002,0.001,0.000,0.002,0.003,0.001,0.004,0.001
max,1.000,0.001,0.001,0.001,0.001,0.001,0.000,0.001,0.001,0.001,...,0.001,0.002,0.002,0.001,0.000,0.002,0.003,0.001,0.004,0.001


In [23]:
# Let's see how many words are in each cluster.
X_train_tfidf_clusters.groupby('clusters').count().sum()

100              2
14th             2
1776             2
1787             2
1917             2
200              2
21st             2
abdicated        2
abhorring        2
abide            2
abiding          2
ability          2
able             2
abraham          2
abroad           2
absolute         2
abuses           2
accept           2
accession        2
accidental       2
accomplished     2
according        2
accordingly      2
account          2
achieve          2
achieved         2
achievement      2
acknowledge      2
acknowledging    2
acquiescence     2
                ..
wish             2
wishes           2
withal           2
withered         2
withhold         2
women            2
wonder           2
wonderful        2
wonders          2
word             2
words            2
work             2
workers          2
working          2
world            2
worldly          2
worse            2
worst            2
worth            2
worthy           2
write            2
written     

# Neural Network

In [24]:
# Establish and fit the multi-level perceptron model.
mlp = MLPClassifier(hidden_layer_sizes=(20,20,), random_state=15)
mlp.fit(X_train_tfidf, ypred)

/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(20, 20), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=15, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [25]:
# Find MLP score.
mlp.score(X_train_tfidf, ypred)

0.9885931558935361

In [26]:
# Find cross-validation score.
cross_val_score(mlp, X_train_tfidf, ypred, cv=5)

/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20

array([0.80909091, 0.82568807, 0.88349515, 0.87254902, 0.88235294])

In [27]:
# Adjust hidden layer parameters.
mlp1 = MLPClassifier(hidden_layer_sizes=(10,10,), random_state=15)
mlp1.fit(X_train_tfidf, ypred)

/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=15, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [28]:
# Find accuracy score.
mlp1.score(X_train_tfidf, ypred)

0.9752851711026616

In [29]:
# Cross-validation.
cross_val_score(mlp1, X_train_tfidf, ypred, cv=5)

/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20

array([0.8       , 0.80733945, 0.86407767, 0.8627451 , 0.88235294])

In [30]:
# Adjust hidden layer parameters.
mlp2 = MLPClassifier(hidden_layer_sizes=(25,15,), random_state=15)
mlp2.fit(X_train_tfidf, ypred)

/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(25, 15), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=15, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [31]:
# Find accuracy score.
mlp2.score(X_train_tfidf, ypred)

0.9923954372623575

In [32]:
# Cross-validation.
cross_val_score(mlp2, X_train_tfidf, ypred, cv=5)

/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20

array([0.80909091, 0.81651376, 0.88349515, 0.87254902, 0.88235294])

# Prepare for Predictive Modeling

In [33]:
# Create baseline score to beat.
# Bush41 had the most sentences, so guessing him
# for all sentences would give this percentage.
print('Baseline score to beat:', sum(
    (sent_df.President == 'bush41') / len(sent_df.President)))

Baseline score to beat: 0.15491452991452967


In [34]:
# Pipeline helpers.
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=15)

In [35]:
# Instantiate the models.
log_reg = LogisticRegression(solver='lbfgs', max_iter=1000)
tree = DecisionTreeClassifier()
forest = RandomForestClassifier()
boost = GradientBoostingClassifier()
nb = BernoulliNB()

In [36]:
# Set up _kwargs files for convenience.
bow_kwargs = {'X_train': X_train_bow, 'y_train': y_train,
              'X_eval': X_eval_bow, 'y_eval': y_eval,
              'X_holdout': X_holdout_bow, 'y_holdout': y_holdout}

tfidf_kwargs = {'X_train': X_train_tfidf_scaled,'y_train': y_train,
                'X_eval': X_eval_tfidf_scaled,'y_eval': y_eval,
                'X_holdout': X_eval_tfidf_scaled,'y_holdout': y_holdout}

In [37]:
# Tune parameter grids.
log_reg_params = {'model__C': [1]}
tree_params = {'model__criterion': ['gini']}
forest_params = {'model__n_estimators': [100]}
boost_params = {'model__n_estimators': [100]}
nb_params = {'model__alpha': [1]}

In [38]:
# Function to fit and predict all working kernals.


def fit_and_predict(model, params: Dict,
                    X_train: pd.DataFrame,
                    y_train: pd.DataFrame,
                    X_eval: pd.DataFrame,
                    y_eval: pd.DataFrame,
                    X_holdout: pd.DataFrame,
                    y_holdout: pd.DataFrame) -> None:
    """
    Takes an instantiated sklearn model, training data (X_train, y_train), 
    and performs cross-validation and then prints the mean of the cross-
    validation accuracies.
    """
    assert len(X_train) == len(y_train)
    assert len(X_eval) == len(y_eval)
    assert len(X_holdout) == len(y_holdout)
    pipe = Pipeline(steps=[('model', model)])
    clf = GridSearchCV(pipe, cv=skf, param_grid=params, n_jobs=2)
    clf.fit(X_train, y_train)
    print('The mean cross_val accuracy on train is',
          f'{clf.cv_results_["mean_test_score"]}.')
    print('The std of the cross_val accuracy is',
          f'{clf.cv_results_["std_test_score"]}.')
    y_pred = clf.predict(X_eval)
    print(classification_report(y_eval, y_pred))
    print(confusion_matrix(y_eval, y_pred))

## Logistic Regression

### Bag of Words

In [39]:
fit_and_predict(log_reg, params=log_reg_params, **bow_kwargs)

/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


The mean cross_val accuracy on train is [0.39163498].
The std of the cross_val accuracy is [0.03240037].
              precision    recall  f1-score   support

      bush41       0.25      0.68      0.37        25
     clinton       0.25      0.19      0.21        16
  eisenhower       0.39      0.47      0.43        19
         fdr       0.60      0.19      0.29        16
   jefferson       0.50      0.10      0.17        10
     kennedy       0.00      0.00      0.00         9
     lincoln       0.63      0.59      0.61        29
       obama       0.40      0.08      0.13        25
      reagan       0.27      0.38      0.32        24
  washington       0.00      0.00      0.00         3

   micro avg       0.35      0.35      0.35       176
   macro avg       0.33      0.27      0.25       176
weighted avg       0.38      0.35      0.32       176

[[17  2  3  0  0  0  0  0  3  0]
 [ 8  3  0  0  0  1  0  1  3  0]
 [ 7  0  9  0  0  0  0  1  2  0]
 [ 7  0  0  3  0  0  3  0  3  0]
 [ 2

/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Tfidf

In [40]:
fit_and_predict(log_reg, params=log_reg_params, **tfidf_kwargs)

AssertionError: 

## Decision Trees

### Bag of Words

In [ ]:
fit_and_predict(tree, params=tree_params, **bow_kwargs)

### Tfidf

In [ ]:
fit_and_predict(tree, params=tree_params, **tfidf_kwargs)

## Random Forest

### Bag of Words

In [ ]:
fit_and_predict(forest, params=forest_params, **bow_kwargs)

### Tfidf

In [ ]:
fit_and_predict(forest, params=forest_params, **tfidf_kwargs)

## Gradient Boosting Machines

### Bag of Words

In [ ]:
fit_and_predict(boost, params=boost_params, **bow_kwargs)

### Tfidf

In [ ]:
fit_and_predict(boost, params=boost_params, **tfidf_kwargs)

## Naive Bayes

### Bag of Words

In [ ]:
fit_and_predict(nb, params=nb_params, **bow_kwargs)

### Tfidf

In [ ]:
fit_and_predict(nb, params=nb_params, **tfidf_kwargs)